In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt
import seaborn as sb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
print(os.listdir('../input/chicago-food-inspections'))

In [ ]:
df = pd.read_csv('../input/chicago-food-inspections/food-inspections.csv', delimiter=',')

In [ ]:
duplicated = df.duplicated(keep='first')

In [ ]:
np.sum(duplicated)

We have 181 duplicates

In [ ]:
df = df.drop_duplicates()

In [ ]:
len(df.columns)

In [ ]:
df = df[['Inspection ID', 'Inspection Date','Inspection Type', 'Results', 'Violations',
         'Latitude', 'Longitude', 'Location', 'Community Areas', 'Census Tracts', 'Wards', 'Zip Codes']]

In [ ]:
type(df)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

This is a summary showing missing data in every column

In [ ]:
df = df.drop(labels=['Community Areas', 'Census Tracts', 'Wards', 'Zip Codes'], axis=1)

I have dropped the column that are completely empty for other ones, we need to agree how to handle them

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
indices = df['Longitude'].isna()

In [ ]:
type(indices)

In [ ]:
df[indices].head()

In [ ]:
df[indices][['Latitude', 'Location']].isna().sum()

No missing partial locations all the location attributes are missing for the same rows

In [ ]:
df.head()

In [ ]:
df['Results'].unique()

In [ ]:
len(df['Inspection ID'].unique())

In [ ]:
df['Violations'] = df['Violations'].str.lower()
df['Inspection Type'] = df['Inspection Type'].str.lower()
df['Results'] = df['Results'].str.lower()

In [ ]:
inspection_type = df['Inspection Type'].unique()

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    try:
        return SequenceMatcher(None, a, b).ratio()
    except:
        print(a, " ", b)

In [ ]:
inspection_type

In [ ]:
cleanedList = [x for x in inspection_type if str(x) != 'nan']
cleanedList

In [ ]:
import itertools

similar_pairs = []

for pair in itertools.combinations(cleanedList, r=2):
    similarity = similar(*pair)
    if similarity > 0.85:
        print(pair)

Some pairs look really similar

In [ ]:
similar('SFP/Complaint', 'sfp/complaint')

* Note not everything is apparent using this metric

In [ ]:
len(inspection_type)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
import spacy as sp
nlp = sp.load('en_core_web_lg')

In [ ]:
%time
iteratable = itertools.combinations(cleanedList, r=2)
inconsistent_pairs = [pair for pair in iteratable if nlp(pair[0]).similarity(nlp(pair[1])) > 0.8]

In [ ]:
for pair in inconsistent_pairs:
    print(pair)

In [ ]:
violations = df['Violations'].str.lower().unique()
results = df['Results'].unique()

In [ ]:
violations = [x for x in violations if str(x) != 'nan']
results = [x for x in results if str(x) != 'nan']

In [ ]:
print(len(results))
print(len(violations))

In [ ]:
print(results)

In [ ]:
iteratable_violations = itertools.combinations(violations, r=2)
inconsistent_pairs_violations = [pair for pair in iteratable_violations if similar(*pair) > 0.7]
iteratable_results = itertools.combinations(results, r=2)
inconsistent_pairs_results = [pair for pair in iteratable_results if similar(*pair) > 0.7]

In [ ]:
for pair in inconsistent_pairs_violations:
    print(pair)

In [ ]:
for pair in inconsistent_pairs_results:
    print(pair)